### Probability Graph Model Benchmark
Whithin this notebook we will compare technics to evaluate a probability graph model for a given seed article through out its neighbors.

In [1]:
from wikipydia import dict_storage, wikidb, parse, wikisyn
WikisynDict = wikisyn.WikisynDict
from collections import Counter
from tabulate import tabulate
import re
from ThreadPool import ThreadPool
import sys

In [2]:
wiki_db = wikidb.WikiDb()

In [3]:
def print_data(headers, data_list):
    """Function to print data tabulated """
    return print(tabulate(data_list, headers=headers))

In [4]:
last_percent_reported = None
def download_progress_hook(count, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

In [5]:
#Function to download bunch of wikipedia pages at once if they are not present
check_and_download__done = 0
def check_and_download(pages):
    global check_and_download__done
    check_and_download__done = 0
    
    n_tasks = len(pages)
    
    print("Tasks to go: " + str(n_tasks))
    
    # Function to be executed in a thread
    def download_stuff(page):
        global check_and_download__done
        #try:
        wiki_db.get_article_by_href(page)
        #except:
            #print("Failed to get page " + page + ". Timed out.")
        #finally:
        check_and_download__done += 1
        download_progress_hook(check_and_download__done, n_tasks)
            #percentage = round(check_and_download__done * 100 / n_tasks)
            #if percentage % 5 == 0:
                #print("{0}%.....".format(percentage), end="")
            #print("Done " + str(check_and_download__done) + "/" + str(n_tasks))

    # Instantiate a thread pool with 5 worker threads
    pool = ThreadPool(50)

    pool.map(download_stuff, pages)
    pool.wait_completion()
    
    print("\nFinishing downloading. Done tasks: " + str(check_and_download__done) + "/" + str(n_tasks))

In [6]:
def get_href_text_count(href):
    synDict = WikisynDict()
    
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    page_text = wikiart.text()
    
    #Populate syndict
    for href, text in links:
        synDict.add_link(href, text)
        
    #Get matches
    links_score = Counter()
        
    for link_href in synDict.keys():
        for l_text, l_score in synDict[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score

    scores_sum = sum(links_score.values())
            
    #norm_links_scores = list(map(lambda a: (a[0],synDict[a[0]], a[1]/scores_sum), links_score.most_common()))
    norm_links_scores = list(map(lambda a: (a[0], a[1]/scores_sum), links_score.most_common()))
    #norm_links_scores = list(map(lambda a: (a[0],list(map(lambda b: b[0], synDict[a[0]])), a[1]/scores_sum), links_score.most_common()))
    
    return norm_links_scores

In [7]:
def get_node_edges_scores(page_href, top=-1):
    """Function to get node edges to be placed in the graph. """
    
    #edges = dict()
    edges = Counter()
    
    #wikiart = get_wiki_article_by_href(page_href)
    wikiart, _ = wiki_db.get_article_by_href(page_href)
    
    #Get main page data
    page_id = wikiart.page_id()
    page_title = wikiart.title()
    #page_links = get_links_score(page_href).items()
    page_links = get_href_text_count(page_href)
    
    if top > -1:
        sorted_page_links = sorted(page_links, key=lambda a: a[1], reverse=True)
        page_links = sorted_page_links[:top]
        #print(page_links)
        
    #Ensure everything has been downloaded first    
    hrefsToDownload = [link_href for link_href, _ in page_links]        
    check_and_download(hrefsToDownload)
    
    #Get links score sum
    score_sum = 0
    for _, score in page_links:
        score_sum += score
    
    for i, (link_href, score) in enumerate(page_links):
        
        #link_art = get_wiki_article_by_href(link_href)
        link_art, _ = wiki_db.get_article_by_href(link_href)
        
        link_id = link_art.page_id()
        link_title = link_art.title()
        
        #If there is already a title already place, sum the scores
        edges[(page_title, link_title)] += score / score_sum
        
    return edges

In [21]:
edges_score = get_node_edges_scores("TensorFlow", 20)
wiki_db.save()
print_data(["Link", "Score"], sorted(edges_score.items(), key=lambda a: a[1], reverse=True))

READ hidden markov models
watch reinforcement
implement the rest of the thing

Tasks to go: 20
5%10%15%20%25%35%40%30%60%50%65%70%45%55%75%85%90%95%100%
Finishing downloading. Done tasks: 20/20
Link                                                  Score
------------------------------------------------  ---------
('TensorFlow', 'Open-source model')               0.226415
('TensorFlow', 'Artificial neural network')       0.188679
('TensorFlow', 'Machine learning')                0.113208
('TensorFlow', 'Deep learning')                   0.0566038
('TensorFlow', 'Google Brain')                    0.0566038
('TensorFlow', 'C++')                             0.0377358
('TensorFlow', 'RankBrain')                       0.0377358
('TensorFlow', 'Supervised learning')             0.0377358
('TensorFlow', 'Jeff Dean (computer scientist)')  0.0377358
('TensorFlow', 'Python (programming language)')   0.0377358
('TensorFlow', 'Tensor processing unit')          0.0377358
('TensorFlow', 'MacOS')                           0.0188679
('TensorFlow', 'Neuroph')                       